In [42]:
import cirq
import numpy as np
from cirq.circuits import InsertStrategy
from cirq          import H, X, SWAP, CZPowGate

In [43]:
## Lets start by creating the gates that we are going to need for our QPE cirquit 

#First we're going to need an IQFT so lets begin by modifying the code from the notes 

# Define the inverse QFT function
def make_IQFT(n, qubits, circuit):
    '''
    Generates an Inverse Quantum Fourier Transform
    :param n: the number of qubits in the cirquit 
    :param qubits: a list containing the values of the qubits 
    :param circuit: the circuit that we are applying the IQFT to 
    :return: circuit with IQFT appended to it 
     '''
    # Swap the qubits as required by QFT
    for i in range(n // 2):
        circuit.append(cirq.SWAP(qubits[i], qubits[n - i - 1]), strategy=cirq.InsertStrategy.NEW)

    # For each qubit apply the appropriate phase
    for i in range(n - 1, -1, -1):
        # Apply CR_k gates where j is the control and i is the target
        k = n - i  # We start with k = n-i
        for j in range(n - 1, i, -1):
            # Define and apply CR_k gate
            crk = cirq.CZPowGate(exponent=-2 / 2**(k))  # Note the i in the exponential is encoded in the gates
            circuit.append(crk(qubits[j], qubits[i]), strategy=cirq.InsertStrategy.NEW)  # Use NEW insert strategy
            k -= 1  # Decrement at each step

        # Apply Hadamard to the qubit
        circuit.append(cirq.H(qubits[i]), strategy=cirq.InsertStrategy.NEW)

# Create the control-U operator
def make_U(phi):
    '''
    Create the U operator by introduing a p[hase shift on the | 1 > state 
    :param phi: The desired phase shift 
    :return: the U operator 
    '''
    
    CU = cirq.CZPowGate(exponent=phi*2)
    return CU

# Quantum Phase Estimation Circuit
def make_QPE(n, j):
    '''
    Create the Quantum Phase Estimation Circuit 
    
    :param n: the amount of decimal places we are aproximaing to and j is the phase we are approximating
    :param j: j the number we are estimating 
    :return: The circuit that performs the phase estimation
    '''

    
    # Begin by creating the circuit, and the qubits
    circuit = cirq.Circuit()
    control_qubits = [cirq.LineQubit(i) for i in range(n)]
    target_qubit = cirq.LineQubit(n)  # Single target qubit

    # Set the target qubit to |1> state
    circuit.append(cirq.X(target_qubit))

    # Apply a Hadamard to each of the control qubits
    circuit.append(cirq.H.on_each(control_qubits))

    # Apply the CU gates to the circuit
    CU = make_U(j)
    for i in range(n):
        # Update the power of CU gate
        CUi = CU**(2**i)
        # Apply CUi gate where n-i-1 is the control
        circuit.append(CUi(control_qubits[n - i - 1], target_qubit))

    # Apply the IQFT to return to the computational basis
    make_IQFT(n, control_qubits, circuit)
    

    return circuit

# Now that we have our circuit, initialize the details and print the circuit
n = 3  #Use small n to display ciurcuit 
j = 0.25  

QPEcircuit = make_QPE(n, j)
print(QPEcircuit)

    
    

0: ───H───────────────@─────×────────────────────@─────────@────────H───
                      │     │                    │         │
1: ───H───────────@───┼─────┼───────@────────H───┼─────────@^-0.5───────
                  │   │     │       │            │
2: ───H───@───────┼───┼─────×───H───@^-0.5───────@^-0.25────────────────
          │       │   │
3: ───X───@^0.5───@───@^0───────────────────────────────────────────────


In [44]:
# Now that we demonstrated that our code indeed reproduces the cirquit for a QPE lets use it to estimate j 
j = 0.7863
n = 5 # we are estimating to 5 decimal places 


#Create the circuit 
QPEcircuit2 = make_QPE(n, j)

#Measure the circuit results 
def measure(circuit,control_qubits):
    '''
    measure measures the circuit and returns the highest probability estimation
    :param circuit: the circuit you wish to measure
    :param control_qubits: the control qubits of the cirquit 
    :return: the highest probability estimation of j 
    '''
    circuit.append(cirq.measure(*control_qubits, key = 'result'))

    #Simulate the results several times to obtain a good sample size 
    s = cirq.Simulator()
    samples = s.run(circuit, repetitions = 1000)
    
    #Print the highest probability estimation
    frequency = list(samples.histogram(key = 'result').keys())[0]
    result = frequency/2**n
    
    return result
result = measure(QPEcircuit2, control_qubits=[cirq.LineQubit(i) for i in range(n)])
print("When estimating", j ,"to",n,"decimal places the estimate is", result)

When estimating 0.7863 to 5 decimal places the estimate is 0.78125


In [45]:
## For fun lets look at how the estimation evolves as we increase n 
for n in range(1,9):
    QPEcircuit3 = make_QPE(n, j)
    result2 = measure(QPEcircuit3,control_qubits=[cirq.LineQubit(i) for i in range(n)])
    print("When estimating", j ,"to",n,"decimal places the estimate is", result2)
    

When estimating 0.7863 to 1 decimal places the estimate is 0.5
When estimating 0.7863 to 2 decimal places the estimate is 0.75
When estimating 0.7863 to 3 decimal places the estimate is 0.75
When estimating 0.7863 to 4 decimal places the estimate is 0.75
When estimating 0.7863 to 5 decimal places the estimate is 0.78125
When estimating 0.7863 to 6 decimal places the estimate is 0.78125
When estimating 0.7863 to 7 decimal places the estimate is 0.8046875
When estimating 0.7863 to 8 decimal places the estimate is 0.78515625
